In [1]:
import nltk
import torch


nltk.download('punkt')
nltk.download('punkt_tab')


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nSetup Complete!")
print(f"PyTorch Version: {torch.__version__}")
print(f"Device Available: {device} (If 'cuda', your training will be super fast)")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.



Setup Complete!
PyTorch Version: 2.9.0+cpu
Device Available: cpu (If 'cuda', your training will be super fast)


## NLTK_UTILS

in this section i make a file named nltk_utils.py in which i define some functions that are necessaey for preprocessing of data in making of a chatbot .
these functions includes tokenization,stemming and bag of words function.

In [2]:
%%writefile nltk_utils.py

# nltk.utils
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer


stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)


def stem(word):
  return stemmer.stem(word.lower())



def bag_of_words(tokenized_sentence,word):
  tokenized_sentence=[stem(w) for w in tokenized_sentence]
  bag=np.zeros(len(word),dtype=np.float32)

  for idx,word in enumerate(word):
    if word in tokenized_sentence:
      bag[idx]=1.0

  return bag

Writing nltk_utils.py


## MODEL(brain):

in this section i make brain for my chatbot named neural_network which includes three layers .first layer includes input_size and hidden_size.second layer is hidden layer which includes hidden size and third layers includes hidden size and num clases.and then i define a forward function that forwards the output of brain to next section

In [3]:
%%writefile model.py
import torch.nn as nn

class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()

    self.l1=nn.Linear(input_size,hidden_size)


    self.l2=nn.Linear(hidden_size,hidden_size)


    self.l3=nn.Linear(hidden_size,num_classes)



    self.relu=nn.ReLU()


  def forward(self,x):
    out=self.l1(x)


    out=self.relu(out)


    out=self.l2(out)

    out=self.relu(out)

    out=self.l3(out)


    return out


Writing model.py


## Training section:

this section named train.py which is most important section for a chatbot .

in this section we connect all brains and bodies of a chatbot together to make it working.in this section i import brain and body from model.py and nltk_utils.py and connect them togatrher by using training loop and its respective functions such as optimizer step and loss criterion and other functions.

In [4]:
%%writefile train.py
import json
import torch.nn as nn
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)


all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))


ignore_words = ['?', '!', ',', '.', '']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))


X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


class ChatDataSet(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples


batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataSet()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)


        labels = labels.to(dtype=torch.long).to(device)


        outputs = model(words)


        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final Loss: {loss.item():.4f}')



data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'Training complete. file saved to {FILE}')

Writing train.py


In [6]:
!python train.py

Epoch [100/1000], Loss: 1.4145
Epoch [200/1000], Loss: 0.4304
Epoch [300/1000], Loss: 0.0793
Epoch [400/1000], Loss: 0.0202
Epoch [500/1000], Loss: 0.0084
Epoch [600/1000], Loss: 0.0045
Epoch [700/1000], Loss: 0.0023
Epoch [800/1000], Loss: 0.0017
Epoch [900/1000], Loss: 0.0008
Epoch [1000/1000], Loss: 0.0010
Final Loss: 0.0010
Training complete. file saved to data.pth
